# Deploys model to KFServing
Note: As this is running inside a KF Pipeline you need to grant additional permissions:

kubectl create clusterrolebinding pipeline-runner-extend --clusterrole cluster-admin --serviceaccount=kubeflow:pipeline-runner (Kudos to Tommy Li https://github.com/Tomcli)

and also inject the storage initializer into the control plane namespace
https://github.com/kubeflow/kfserving/issues/1389

In [ ]:
!pip3 install kfp-tekton
!pip3 install kfserving==0.4.1

In [ ]:
from kfserving import constants
from kfserving import KFServingClient
from kfserving import utils
from kfserving import V1alpha2EndpointSpec
from kfserving import V1alpha2InferenceService
from kfserving import V1alpha2InferenceServiceSpec
from kfserving import V1alpha2PredictorSpec
from kfserving import V1alpha2TensorflowSpec
from kubernetes import client
from kubernetes.client import V1ResourceRequirements
import os

In [ ]:
# @param model_name How the model should be named on KFServing
# @param model_uri URI where the model can be downloaded from

In [ ]:
model_name = os.environ.get('model_name')
model_uri = os.environ.get('model_uri')

In [ ]:
constants.KFSERVING_VERSION

Define namespace where InferenceService needs to be deployed to. If not specified, below function defines namespace to the current one where SDK is running in the cluster, otherwise it will deploy to default namespace.

In [ ]:
namespace = utils.get_default_target_namespace()

## Define InferenceService

Firstly define default endpoint spec, and then define the inferenceservice basic on the endpoint spec.

In [ ]:
api_version = constants.KFSERVING_GROUP + '/' + constants.KFSERVING_VERSION
default_endpoint_spec = V1alpha2EndpointSpec(
    predictor=V1alpha2PredictorSpec(
        tensorflow=V1alpha2TensorflowSpec(
            storage_uri=model_uri,
            resources=V1ResourceRequirements(
                requests={'cpu': '100m', 'memory': '1Gi'},
                limits={'cpu': '100m', 'memory': '1Gi'}))))

isvc = V1alpha2InferenceService(
    api_version=api_version,
    kind=constants.KFSERVING_KIND,
    metadata=client.V1ObjectMeta(
        name='flower-sample',
        namespace=namespace
    ),
    spec=V1alpha2InferenceServiceSpec(default=default_endpoint_spec)
)

In [ ]:
KFServing = KFServingClient()
KFServing.delete(model_name, namespace=namespace)

## Create InferenceService

Call KFServingClient to create InferenceService.

In [ ]:
KFServing = KFServingClient()
KFServing.create(isvc)

## Check the InferenceService

In [ ]:
KFServing.get(model_name, namespace=namespace, watch=True, timeout_seconds=120)